In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")
plt.rcParams['figure.figsize'] = 16, 12
import pandas as pd
from tqdm import tqdm_notebook
import io
from PIL import Image
from glob import glob
from collections import defaultdict
import os
import pickle
import cv2
import math
import json
import itertools as it

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim

from kaggle_camera_model_id_lib.utils import PechkaBot, ImageList, NpzFolder, NCrops, TifFolder, MultiDataset
from kaggle_camera_model_id_lib.models import VggHead, StyleVggHead, IEEEfcn, ResNetFC, ResNetX, FatNet1
from kaggle_camera_model_id_lib.models import InceptionResNetV2fc, InceptionResNetV2fcSmall
from kaggle_camera_model_id_lib.utils import jpg_compress, equalize_v_hist, hsv_convert
from kaggle_camera_model_id_lib.utils import scale_crop_pad, gamma_correction
from kaggle_camera_model_id_lib.utils import patch_quality_dich, n_random_crops, n_pseudorandom_crops
from kaggle_camera_model_id_lib.models import DANet, ResNetFeatureExtractor, AvgFcClassifier, FCDiscriminator
from kaggle_camera_model_id_lib.models import AvgClassifier

import imgaug as ia
from imgaug import augmenters as iaa

In [2]:
crop_center_size = 1520
crop_size = 256
random_crop = transforms.RandomCrop(crop_size)
center_crop = transforms.CenterCrop(crop_center_size)
rvf = transforms.RandomVerticalFlip()
rhf = transforms.RandomHorizontalFlip()
random_flip = lambda img: rvf(rhf(img))

n_crops_search_train = 36
n_crops_train = 9

to_tensor = transforms.ToTensor()
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

In [3]:
train_list_path = '/home/mephistopheies/storage2/data/camera-model-id/train.tsv'
val_path = '/home/mephistopheies/storage2/data/camera-model-id/val/'
test_path = '/home/mephistopheies/storage2/data/camera-model-id/raw/test/'

In [2]:
X_var = Variable(torch.from_numpy(np.random.uniform(-1, 1, size=(5, 3, 256, 256)).astype(np.float32)))

In [5]:
# n_classes = 10 

# model_fe = ResNetFeatureExtractor(models.resnet.BasicBlock, [3, 4, 6, 3], 
#                                   load_resnet='resnet34')
# model_c = AvgFcClassifier(n_classes)
# model_d = FCDiscriminator()

# model = DANet(model_fe, model_d, model_c)

# print(model(X_var, mode='c'))
# print(model(X_var, mode='d'))

- https://github.com/devnag/pytorch-generative-adversarial-networks/blob/master/gan_pytorch.py
- https://github.com/znxlwm/pytorch-generative-model-collections
- https://github.com/pytorch/examples/blob/master/dcgan/main.py

In [6]:
# step_crop_val = 256

# ds_val = NpzFolder(
#     val_path,
#     transform=transforms.Compose([
#         transforms.Lambda(lambda img: NCrops(img, crop_size=crop_size, step=step_crop_val)),
#         transforms.Lambda(lambda crops: torch.stack([normalize(to_tensor(Image.fromarray(crop)))
#                                                      for crop in crops]))
#     ]),
#     target_transform=transforms.Compose([
#         transforms.Lambda(lambda y: [y]*int(np.floor(1 + (512 - crop_size)/step_crop_val))**2),
#         transforms.Lambda(lambda ylist: torch.LongTensor(ylist))
#     ]))
# val_loader = torch.utils.data.DataLoader(    
#     ds_val,
#     batch_size=10, 
#     shuffle=False,
#     num_workers=1, 
#     pin_memory=True)

In [2]:
model_factory = {
    'resnet34_fe': lambda: ResNetFeatureExtractor(models.resnet.BasicBlock, [3, 4, 6, 3], load_resnet='resnet34'),
    'AvgFcClassifier': lambda n_classes: AvgFcClassifier(n_classes),
    'FCDiscriminator': lambda: FCDiscriminator(),
    'AvgClassifier512': lambda n_classes: AvgClassifier(n_classes, 512)
}

model = DANet(
        model_factory['resnet34_fe'](),
        model_factory['FCDiscriminator'](),
        model_factory['AvgClassifier512'](10)).cuda()

In [13]:
state = model.classifier.state_dict()

In [14]:
len(state.keys())

6

In [7]:
checkpoint = torch.load('/home/mephistopheies/storage2/data/camera-model-id/models/resnet34fc/256_pretrained_random_aug_kaggle_10/best_model.tar')
d = checkpoint['model']
del(checkpoint)

In [11]:
len(d.keys())

186

In [8]:
criterion_dg = nn.BCELoss().cuda()

In [9]:
#     ds_val_c = NpzFolder(
#         val_path,
#         transform=transforms.Compose([
#             transforms.Lambda(lambda img: NCrops(img, crop_size=crop_size, step=step_crop_val)),
#             transforms.Lambda(lambda crops: torch.stack([normalize(to_tensor(aug_optional_val(Image.fromarray(crop)))) 
#                                                          for crop in crops]))
#         ]),
#         target_transform=transforms.Compose([
#             transforms.Lambda(lambda y: [y]*int(np.floor(1 + (512 - crop_size)/step_crop_val))**2),
#             transforms.Lambda(lambda ylist: torch.LongTensor(ylist))
#         ]))
#     val_loader_c = torch.utils.data.DataLoader(    
#         ds_val_c,
#         batch_size=batch_size_val_c, 
#         shuffle=False,
#         num_workers=workers, 
#         pin_memory=True)
#     log('val_loader_c.size: %i' % len(val_loader_c.dataset.imgs))    

In [4]:
crop_size = 256
step_crop_val = 128
batch_size_train_d = 7
workers = 1


ds_train_d_test = TifFolder(
    test_path,
    transform=transforms.Compose([
        transforms.Lambda(lambda img: NCrops(np.array(img), crop_size=crop_size, step=step_crop_val)),
        transforms.Lambda(lambda crops: torch.stack([normalize(to_tensor(crop)) for crop in crops]))
    ]),
    target_transform=transforms.Compose([
        transforms.Lambda(lambda y: [0]*int(np.floor(1 + (512 - crop_size)/step_crop_val))**2),
        transforms.Lambda(lambda ylist: torch.FloatTensor(ylist))
    ]))
train_loader_d_test = torch.utils.data.DataLoader(    
    ds_train_d_test,
    batch_size=batch_size_train_d, 
    shuffle=False,
    num_workers=workers, 
    pin_memory=True)


ds_train_d_val = NpzFolder(
    val_path,
    transform=transforms.Compose([
        transforms.Lambda(lambda img: NCrops(img, crop_size=crop_size, step=step_crop_val)),
        transforms.Lambda(lambda crops: torch.stack([normalize(to_tensor(Image.fromarray(crop)))
                                                     for crop in crops]))
    ]),
    target_transform=transforms.Compose([
        transforms.Lambda(lambda y: [1]*int(np.floor(1 + (512 - crop_size)/step_crop_val))**2),
        transforms.Lambda(lambda ylist: torch.FloatTensor(ylist))
    ]))
train_loader_d_val = torch.utils.data.DataLoader(    
    ds_train_d_val,
    batch_size=batch_size_train_d, 
    shuffle=False,
    num_workers=workers, 
    pin_memory=True)

In [5]:
train_loader_d_test_chain = it.chain(*[train_loader_d_test]*5)
train_loader_d_val_chain = it.chain(*[train_loader_d_val]*5)

X_test, Y_test = train_loader_d_test_chain.__next__()
X_val, Y_val = train_loader_d_val_chain.__next__()

In [6]:
print(X_test.shape)
print(Y_test.shape)
print(X_val.shape)
print(Y_val.shape)

torch.Size([7, 9, 3, 256, 256])
torch.Size([7, 9])
torch.Size([7, 9, 3, 256, 256])
torch.Size([7, 9])


In [13]:
bs, ncrops, c, h, w = X_test.shape
X_test = X_test.view(-1, c, h, w)
Y_test = Y_test.view(ncrops*bs)
X_val = X_val.view(-1, c, h, w)
Y_val = Y_val.view(ncrops*bs)

X = torch.cat([X_test, X_val], dim=0)
Y = torch.cat([Y_test, Y_val], dim=0)

print(X_test.shape)
print(Y_test.shape)
print(X_val.shape)
print(Y_val.shape)

print(X.shape)
print(Y.shape)

torch.Size([63, 3, 256, 256])
torch.Size([63])
torch.Size([63, 3, 256, 256])
torch.Size([63])
torch.Size([126, 3, 256, 256])
torch.Size([126])


In [14]:
X_var = Variable(X.cuda())
Y_var = Variable(Y.cuda())

p = model(X_var, mode='d')

In [24]:
loss = criterion_dg(p.squeeze(), Y_var)

In [32]:
train_list_path = '/home/mephistopheies/storage2/data/camera-model-id/train.tsv'
train_list_pseudo_npz = '/home/mephistopheies/storage2/data/camera-model-id/pseudo_labels/phase1/'

n_crops_train = 5
batch_size_train = 7
workers = 1

def aug_train(img):
    if min(img.size) > crop_center_size:
        return random_flip(random_crop(center_crop(img)))
    img_np = np.array(img)
    if img_np.shape[0] < crop_center_size and img_np.shape[1] > crop_center_size:
        n = np.random.randint(img_np.shape[1] - crop_center_size)
        return random_flip(random_crop(Image.fromarray(img_np[:, n:(n + crop_center_size), :])))
    if img_np.shape[1] < crop_center_size and img_np.shape[0] > crop_center_size:
        n = np.random.randint(img_np.shape[0] - crop_center_size)
        return random_flip(random_crop(Image.fromarray(img_np[n:(n + crop_center_size), :, :])))
    return random_flip(random_crop(img))

transform_train = transforms.Compose([
    transforms.Lambda(lambda img: [
        aug_train(img)
        for i in range(n_crops_train)
    ]),
    transforms.Lambda(lambda crops: torch.stack([normalize(to_tensor(crop)) for crop in crops]))
])

ds_train = ImageList(
    train_list_path,
    transform=transform_train,
    target_transform=transforms.Compose([
        transforms.Lambda(lambda y: [y]*n_crops_train),
        transforms.Lambda(lambda ylist: torch.LongTensor(ylist))
    ]))

    
ds_train_pseudo = NpzFolder(
    train_list_pseudo_npz,
    transform=transforms.Compose([
        transforms.Lambda(lambda img: [
            aug_train(Image.fromarray(img))
            for i in range(n_crops_train)
        ]),
        transforms.Lambda(lambda crops: torch.stack([normalize(to_tensor(crop)) for crop in crops]))
    ]),
    target_transform=transforms.Compose([
        transforms.Lambda(lambda y: [y]*n_crops_train),
        transforms.Lambda(lambda ylist: torch.LongTensor(ylist))
    ]))
# ds_train = MultiDataset([ds_train, ds_train_pseudo])

# train_loader = torch.utils.data.DataLoader(    
#     ds_train,
#     batch_size=batch_size_train, 
#     shuffle=True,
#     num_workers=workers, 
#     pin_memory=True)

In [10]:
ds_train[111][0].shape

torch.Size([5, 3, 256, 256])

In [20]:
# ds_train_pseudo[111][0].shape

In [1]:
# ds_train_pseudo[111][0]